In [3]:
pip install transformers

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB ? eta -:--:--
   -- ------------------------------------- 0.5/10.0 MB 1.9 MB/s eta 0:00:06
   ------- -------------------------------- 1.8/10.0 MB 3.0 MB/s eta 0:00:03
   ------------- -------------------------- 3.4/10.0 MB 4.5 MB/s eta 0:00:02
   ------------------------ --------------- 6.0/10.0 MB 6.1 MB/s eta 0:00:01
   ---------------------------------- ----- 8.7/10.0 MB 7.3 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 7.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------  2.4/2.4 MB 11.2 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 9.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from datasets import Dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
import pandas as pd

d:\app\Python\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
d:\app\Python\Lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvisi

In [ ]:

model = GPT2LMHeadModel.from_pretrained('D:\WorkSpace\Source\webbanhang\AIModule\FineturingModel\V2')
tokenizer = GPT2Tokenizer.from_pretrained('D:\WorkSpace\Source\webbanhang\AIModule\FineturingModel\V2')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
model.eval()


cpu


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [86]:
import re
def remove_sentences_with_patterns(text):
    # Các regex để phát hiện email, số điện thoại, và URL
    email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b"
    phone_pattern = r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}|\d{4}[-\.\s]??\d{3}[-\.\s]??\d{3})'
    url_pattern = r"\b(?:https?://|www\.)\S+\b"
    
    # Tách văn bản thành các câu
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    # Giữ lại các câu không khớp với bất kỳ mẫu nào
    filtered_sentences = [
        sentence for sentence in sentences
        if not (re.search(email_pattern, sentence) or 
                re.search(phone_pattern, sentence) or 
                re.search(url_pattern, sentence))
    ]
    print(filtered_sentences)
    # Gộp lại thành văn bản sau khi lọc
    return ' '.join(filtered_sentences)

In [85]:
remove_sentences_with_patterns('''
Hỏi: chào bạn Trả lời: Chào bạn, bạn có thể tham khảo một số mẫu thiết kế biệt thự đẹp mà chúng tôi gợi ý dưới đây. (Zalo, Viber): 0909.599.490 (Mr. Thiện,56568, viber, zalo, facebook)
Đáp: Cảm ơn bạn.
['Hỏi: chào bạn Trả lời: Chào bạn, bạn có thể tham khảo một số mẫu thiết kế biệt thự đẹp mà chúng tôi gợi ý dưới đây.', '(Zalo, Viber): 0909.599.490 (Mr.', 'Thiện, 0918.248.278, viber, zalo, facebook)\nĐáp: Cảm ơn bạn']''')

['\nHỏi: chào bạn Trả lời: Chào bạn, bạn có thể tham khảo một số mẫu thiết kế biệt thự đẹp mà chúng tôi gợi ý dưới đây.', 'Thiện,56568, viber, zalo, facebook)\nĐáp: Cảm ơn bạn.']


'\nHỏi: chào bạn Trả lời: Chào bạn, bạn có thể tham khảo một số mẫu thiết kế biệt thự đẹp mà chúng tôi gợi ý dưới đây. Thiện,56568, viber, zalo, facebook)\nĐáp: Cảm ơn bạn.'

In [30]:

text = 'Mua quà gì cho con gái tôi mặc là đẹp đây?';
input_text = f"Hỏi: {text} Trả lời:"
inputs = tokenizer(input_text, return_tensors="pt").to(device)



model.eval()
outputs = model.generate(
    inputs['input_ids'],
    pad_token_id=tokenizer.pad_token_id,
    do_sample=True,
    max_length=64,
    min_length=10,
    top_k=40,
    num_beams=5,
    early_stopping=True,
    no_repeat_ngram_size=2,
    num_return_sequences=1
)


generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
for i, text in enumerate(generated_texts):
    text = text.split('|')[0]
    text = text.split('.')
    text = text[:2]
    text = [t for t in text if 'http' not in t]
    text = '.'.join(text)
    print(f"Generated Text {i+1}:\n{text}.")


Generated Text 1:
Hỏi: Mua quà gì cho con gái tôi mặc là đẹp đây? Trả lời: Bạn có thể chọn một bộ đầm hoặc một chiếc váy dài để tạo sự thoải mái cho bé. Một chiếc áo sơ mi hoặc áo khoác nhẹ sẽ giúp bé dễ dàng di chuyển.
